# workflow
01. 确定研究区
02. 研究时间段，历史洪水区
03. 获取影像，评估数据数量和质量
04. 提取时间序列水域
05. 精度评价

In [ ]:
import ee, os, geemap
import numpy as np
import pandas as pd

ee.Initialize()
Map = geemap.Map()

In [ ]:
bound_roi = ee.FeatureCollection('users/yehuigeo/poyanghu')
date_yr = 1997, 2020
doy = '-01-01', '-12-31'
date_st = str(date_yr[0]) + doy[0]
date_ed = str(date_yr[1]) + doy[1]

In [ ]:
# cor_list = bound_roi.geometry().coordinates().getInfo()

In [ ]:
# get corner points
# cors = np.array(cor_list[0])
# x1 = cors[:,:,0].min()
# x2 = cors[:,:,0].max()
# y1 = cors[:,:,1].min()
# y2 = cors[:,:,1].max()

In [ ]:
roi = bound_roi.union().geometry().buffer(5000).bounds()

##符合质量的影像数据日期
Landsat-5卫星 1984-03-01 - 2008-07-29
Landsat-7卫星 1999-4-15
Landsat-8卫星 2013-2-11


In [ ]:
# -------------去云----------------------------------
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2).divide(10000)

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    # Get the pixel QA band.
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0)).divide(10000)
    return image.updateMask(mask)


归一化水指数
NDWI = (绿波段 - 近红外波段) / (绿波段 + 近红外波段)

landsat8: NDWI = (band3 - band5) / (band3 + band5)
landsat5/7: NDWI = (band2 - band4) / (band2 + band4)
sentinel2: NDWI = (band3 - band8) / (band3 + band8)

-------------------------------------------------------
<!-- Landsat7 band introduction
Band 1 (blue) 
Band 2 (green) 
Band 3 (red) 
Band 4 (near infrared) 
Band 5 (shortwave infrared 1)
Band 6 briahtness temoerature
Band 7 (shortwave infrared 2） -->

In [ ]:
def cal_EVI(img):
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select('B4'),
          'RED': img.select('B3'),
          'BLUE':img.select('B1')})
    return EVI.rename('EVI').updateMask(EVI.gt(-1).And(EVI.lt(1)))
def cal_GCVI(img):
    GCVI = img.expression(
        '(NIR / Green - 1)', {
          'NIR': img.select('B4'),
          'Green': img.select('B2')})
    return GCVI.rename('GCVI')
def cal_LSWI(img): #  the shortwave infrared (SWIR) and the NIR regions of the electromagnetic spectrum
    LSWI = img.normalizedDifference(['B4','B5']).rename('LSWI')
    return LSWI.updateMask(LSWI.gt(-1).And(LSWI.lt(1)))
def cal_NDWI(img):
    NDWI = img.normalizedDifference(['B2','B4']).rename('NDWI')
    return NDWI.updateMask(NDWI.gt(-1).And(NDWI.lt(1)))
def cal_mNDWI(img):
    mNDWI = img.normalizedDifference(['B2','B5']).rename('mNDWI')
    return mNDWI.updateMask(mNDWI.gt(-1).And(mNDWI.lt(1)))
def cal_NDVI(img):
    NDVI = img.normalizedDifference(['B4','B3']).rename('NDVI')
    return NDVI.updateMask(NDVI.gt(-1).And(NDVI.lt(1)))
# def cal_NDSI(img):
#     NDSI = img.normalizedDifference(['B2','B5']).rename('NDSI')
def addVIs(img):
    b1 = cal_NDVI(img)
    b2 = cal_EVI(img)
    b3 = cal_LSWI(img)
    b4 = cal_NDWI(img)
    b5 = cal_mNDWI(img)
    VIs_imgs = img.addBands(b1).addBands(b2).addBands(b3).addBands(b4).addBands(b5)
    return VIs_imgs
# def addVIs(img):
#     NDWI = img.normalizedDifference(['B3','B5']).rename('NDWI')
#     LSWI = img.normalizedDifference(['B5','B6']).rename('LSWI')
#     EVI = f_EVI(img)
#     GCVI = f_GCVI(img)
#     return img.addBands(NDVI).addBands(NDWI).addBands(LSWI).addBands(EVI).addBands(GCVI)

<!-- Landsat7 band introduction
Band 1 (blue) 
Band 2 (green) 
Band 3 (red) 
Band 4 (near infrared) 
Band 5 (shortwave infrared 1)
Band 6 briahtness temoerature
Band 7 (shortwave infrared 2） -->

In [ ]:
# ------------------get imageCollection data---------------------------------
collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .filterBounds(bound_roi) \
    .filterDate(date_st, date_ed) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
# print('Landsat5')
print(collection5.size().getInfo())
# get landsat 7
collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .filterBounds(bound_roi) \
    .filterDate(date_st, date_ed) \
    .map(cloudMaskL457) \
    .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])
# get landsat 8
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(bound_roi) \
    .filterDate(date_st, date_ed) \
    .map(maskL8sr).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']) 
print(collection8.size().getInfo())
# get sentinel-2A
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate(date_st, date_ed).map(maskS2clouds).filterBounds(bound_roi)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) 

In [ ]:
class down_para:
    dir_save = r'D:\03.Temporary'
    f_format = '.tif'
if not os.path.exists(down_para.dir_save):
    os.makedirs(down_para.dir_save)

In [ ]:
date_list5 = collection5.aggregate_array('system:index').getInfo()
date_list7 = collection7.aggregate_array('system:index').getInfo()


In [ ]:
date_list8 = collection8.aggregate_array('system:index').getInfo()

In [ ]:
# px = collection5.first().get('system:index').getInfo()
# def class_images(i):
#     [types, locates, doys] = i.split('_')
def get_images_info(date_list):
    for i, j in enumerate(date_list):  
        arr_info = np.array(j.split('_')).reshape(1,-1)
        if i == 0:
            arr_stas = arr_info
            continue
        arr_stas = np.vstack([arr_stas,arr_info])
    df_images_info = pd.DataFrame(arr_stas, columns=['type', 'locate', 'date'])    
    return df_images_info           
#         image_info = np.array(i.split('_')).reshape(1,-1)

In [ ]:
df_info_col8 = get_images_info(date_list8)
# print(df_info_col8)
# print(df_images_info['date'])
# df_images_info = pd.DataFrame(arr_stas, columns=['type', 'locate', 'date']) 

In [ ]:
df_info_col7 = get_images_info(date_list7)
df_info_col5 = get_images_info(date_list5)
# print(df_info_col5)
# print(df_info_col7)

计算多种指数提取水域，水域汛期规律

In [ ]:
def water_detect(img):
    c1 = img.select('mNDWI')
    c2 = img.select('EVI')
    c3 = img.select('NDVI')
    img_water = c1.gt(c2).Or(c1.gt(c3)).And(c2.lt(0.1))
    return img_water
# def NonWater_detect(img):
#     img_NonWater = img.select('mNDWI').lte(img.select('EVI')).and(img.select('mNDWI').lte(img.select('NDVI')).or(img.select('EVI').gte(0.1))
#     return img_NonWater

In [ ]:
images_water = collection5.map(cal_mNDWI).select('mNDWI').mean()

In [ ]:
# print(images_water.getInfo())
Map.addLayer(images_water.clip(bound_roi), {}, "show1")

In [ ]:
Map.addLayer(images_water.clip(bound_roi), {}, "show1")
Map.centerObject(roi)
Map

In [ ]:
file = down_para.dir_save + os.sep + 'test11' + down_para.f_format
# 导出栅格影像，file_per_band为是否按波段分别导出
geemap.ee_export_image(images_water.clip(roi), filename=file, scale=30,region=roi,file_per_band=True)